In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import os
import itertools
from collections import OrderedDict
from functools import partial

from sklearn.datasets import load_svmlight_file, make_circles


from links import LinksClassifier
from logit import LogisticRegressionPairwise, LogisticRegression


from sklearn.model_selection import ParameterGrid, StratifiedShuffleSplit, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

from tqdm import tqdm_notebook as tqdm

from new_experiment_runner.cacher import CSVCacher

In [3]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

In [5]:
params = pd.read_csv('data/params.csv')

In [7]:
params.head()

,cv_split,gs_test_size,cv_splits,cv_test_size,percent_unlabeled,percent_links,dataset,beta,delta,cv_random_state,gs_splits,percent_labels,alpha,cv_score,gamma,test_score
0,0.0,0.2,3.0,0.2,0.3,0.3,diabetes_scale,0.2,0.01,42.0,3.0,0.3,10.0,0.738739,1.0,0.681818
1,0.0,0.2,3.0,0.2,0.3,0.3,diabetes_scale,0.1,0.40,42.0,3.0,0.3,10.0,0.738739,0.5,0.701299
2,2.0,0.2,3.0,0.2,0.3,0.3,diabetes_scale,0.1,0.50,42.0,3.0,0.3,10.0,0.765766,0.5,0.746753
3,0.0,0.2,3.0,0.2,0.3,0.3,diabetes_scale,0.1,0.30,42.0,3.0,0.3,10.0,0.738739,1.0,0.688312
4,0.0,0.2,3.0,0.2,0.3,0.3,diabetes_scale,0.1,0.01,42.0,3.0,0.3,10.0,0.738739,0.5,0.694805


In [16]:
def plot_scores(ax, scores, vmin, vmax, range_x, range_y):
    r = ax.imshow(scores, 
        interpolation='nearest',
                  cmap=plt.cm.hot,
                  vmax=vmax,
                  vmin=vmin,
                origin='lower')
    ax.set_xticks(np.arange(len(range_x)))
    ax.set_xticklabels(range_x)
    ax.set_yticks(np.arange(len(range_y)))
    ax.set_yticklabels(range_y)
    return r

In [34]:
%matplotlib tk

In [29]:
params.dataset.unique()

array(['diabetes_scale', 'breast-cancer_scale', 'australian_scale',
       'circles', 'moons'], dtype=object)

In [39]:
datasets =params.dataset.unique()[:3]
fig, ax = plt.subplots(ncols=len(datasets))

for i, ds in enumerate(datasets):
    ds_params = params.ix[params['dataset'] == ds]
    ds_score_grouped = pd.groupby(ds_params, by=['beta', 'delta']).agg({'test_score':np.mean})
    ds_score_grouped = ds_score_grouped.unstack(level=-1)
    
    plot_scores(ax[i], ds_score_grouped, vmin=0.5, vmax=1, range_y=np.sort(ds_params.beta.unique()),
                range_x=np.sort(ds_params.delta.unique()))
    

Float64Index([0.01, 0.05, 0.1, 0.15, 0.2, 0.3, 0.4, 0.5], dtype='float64', name=u'delta')

In [ ]:
imsh